<a href="https://colab.research.google.com/github/khanabdulmajid/tensorflow_deep_learning/blob/main/Universal_Sentence_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using USE in a tf model

In [1]:
!wget https://raw.githubusercontent.com/khanabdulmajid/tensorflow_deep_learning/main/helper_functions.py

--2023-10-14 12:16:32--  https://raw.githubusercontent.com/khanabdulmajid/tensorflow_deep_learning/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-10-14 12:16:32 (46.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [27]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [8]:
import tensorflow as tf
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub



In [3]:
sentence_encoder_layer=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                      input_shape=[],
                                       dtype=tf.string,
                                      trainable=False,
                                      name="USE")


In [5]:
# x=sentence_encoder_layer
# outputs= tf.keras.layers.Dense(1,activation="sigmoid")(x)
# model_USE=tf.keras.Model(x,outputs)

In [6]:
 !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("/content/nlp_getting_started.zip")

--2023-10-14 12:23:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.206.207, 142.250.159.207, 142.251.120.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.206.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-10-14 12:23:43 (155 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [9]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
#shuffle training_data
train_df_shuffled=train_df.sample(frac=1,random_state=42) #frac--> what percent of data
train_df_shuffled.head()
train_sentences,val_sentences,train_labels, val_labels =  train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                         train_df_shuffled["target"].to_numpy(),
                                                                         test_size=0.1,
                                                                         random_state=42
                                                                         )

#SEQUENTIAL MODEL

In [45]:
model_use_sequential=tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [46]:
model_use_sequential.compile(loss="binary_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=["accuracy"]
                  )


In [47]:
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath="content/modelCheckpoint",
                                                    save_best_only=True,
                                                    save_weights_only=True,
                                                    monitor="val_accuracy")

In [ ]:
model_use_history=model_use_sequential.fit(
    x=train_sentences,
    y=train_labels,
    epochs=15,
    validation_data=(val_sentences,val_labels),
    callbacks=[create_tensorboard_callback(dir_name="Callbacks",experiment_name="Universal_Sentence_Encoder_1"),
               model_checkpoint]
)

Saving TensorBoard log files to: Callbacks/Universal_Sentence_Encoder_1/20231014-131932
Epoch 1/15
215/215 [==============================] - 16s 56ms/step - loss: 0.4856 - accuracy: 0.7705 - val_loss: 0.4409 - val_accuracy: 0.8071
Epoch 2/15
215/215 [==============================] - 9s 43ms/step - loss: 0.3990 - accuracy: 0.8259 - val_loss: 0.4213 - val_accuracy: 0.8097
Epoch 3/15
215/215 [==============================] - ETA: 0s - loss: 0.3735 - accuracy: 0.8397

In [ ]:
plot_loss_curves(model_use_history)

In [42]:
pred_probs=tf.round(tf.squeeze(model_use_sequential.predict(val_sentences)))

24/24 [==============================] - 1s 15ms/step


In [43]:
model_2_results=calculate_results(y_true=val_labels,y_pred=pred_probs)

In [44]:
model_2_results

{'accuracy': 81.75853018372703,
 'precision': 0.8202616926815424,
 'recall': 0.8175853018372703,
 'f1': 0.8159820561172786}